In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
cp /content/drive/MyDrive/base_model/kvasir-dataset-v2.zip .

In [ ]:
! 7z x kvasir-dataset-v2.zip -aoa -o/content/

In [ ]:
import os
import numpy as np
import seaborn as sea
import tensorflow as tf 
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow.keras.applications as Applications
import tensorflow.keras.preprocessing.image as ImagePreprocessing
import cv2
import pickle
import json
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Dense ,MaxPool2D ,BatchNormalization,Flatten ,Dropout
from sklearn.model_selection import KFold
from tensorflow.keras.applications.resnet50 import ResNet50
from PIL import Image
import importlib.util
import random as rn
import pandas as pd
from sklearn.model_selection import train_test_split
from shutil import copyfile
from matplotlib import pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50
import json
import tensorflow.keras as keras

sns.set()

In [ ]:
train_split = 0.7
val_split = 0.15
test_split = 0.15
original_ds_path = 'kvasir-dataset-v2'
train_ds_path = 'train'
val_ds_path = 'validate'
test_ds_path = 'test'
base_dir = '/content/drive/MyDrive/base_model/'
losses_dir = base_dir+'losses/'
model_save= base_dir+'model_saves/'
seed = 49
size_img = (224,224) #size choosen to preserve the aspect ratio
input_size = (224,224,3)
batch_size = 16

In [ ]:
ds_dic = {}
for folder in os.listdir(original_ds_path):
  ds_dic[folder] = os.listdir(original_ds_path+'/'+folder) 

In [ ]:
if not os.path.exists(train_ds_path):
      os.mkdir(train_ds_path)
if not os.path.exists(val_ds_path):
      os.mkdir(val_ds_path)
if not os.path.exists(test_ds_path):
      os.mkdir(test_ds_path)

for each in ds_dic:
    
    if not os.path.exists(train_ds_path+'/'+each+'/'):
        os.mkdir(train_ds_path+'/'+each+'/')
    if not os.path.exists(val_ds_path+'/'+each+'/'):
        os.mkdir(val_ds_path+'/'+each+'/')
    if not os.path.exists(test_ds_path+'/'+each+'/'):
        os.mkdir(test_ds_path+'/'+each+'/')

In [ ]:
for each in ds_dic.keys():
  train_split , intermediate_split=train_test_split(ds_dic[each], 
                                              test_size=0.3, 
                                              random_state=seed,
                                              shuffle = True)
  
  val_split , test_split = train_test_split(intermediate_split, 
                                              test_size=0.5, 
                                              random_state=seed,
                                              shuffle = True)
  



  for img in train_split:
        destination = train_ds_path+'/'+each+'/'+img
        source = original_ds_path+'/'+each+'/'+img
        
        if not os.path.exists(destination):
            copyfile(source, destination)
        
  for img in test_split:
      destination = test_ds_path+'/'+each+'/'+img
      source = original_ds_path+'/'+each+'/'+img
      
      if not os.path.exists(destination):
          copyfile(source, destination)

  for img in val_split:
      destination = val_ds_path+'/'+each+'/'+img
      source = original_ds_path+'/'+each+'/'+img
      
      if not os.path.exists(destination):
          copyfile(source, destination)

In [ ]:
train_datagen = ImageDataGenerator(featurewise_center=True,
                               samplewise_center=True,
                               featurewise_std_normalization=True,
                               samplewise_std_normalization=True,
                               zca_whitening=False,
                               zca_epsilon=1e-06,
                               rotation_range=180,
                               width_shift_range=0.0,
                               height_shift_range=0.0, 
                               brightness_range=None, 
                               shear_range=0.0, 
                               zoom_range=[0.0,2.0],
                               channel_shift_range=0.0, 
                               fill_mode='nearest', 
                               cval=0.0, 
                               horizontal_flip=True,
                               vertical_flip=True, 
                               rescale=1./255, 
                               preprocessing_function=None,
                               validation_split=0.15
    )

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(directory=train_ds_path,
                                                    target_size=size_img,
                                                    batch_size=batch_size,
                                                    shuffle=True, 
                                                    seed=seed,
                                                    )
val_generator = val_datagen.flow_from_directory(directory=val_ds_path,
                                                    target_size=size_img,
                                                    batch_size=batch_size,
                                                    shuffle=True, 
                                                    seed=seed,
                                                    )
test_generator = test_datagen.flow_from_directory(directory=test_ds_path,
                                                    target_size=size_img,
                                                    batch_size=batch_size,
                                                    shuffle=True, 
                                                    seed=seed,
                                                    )

In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


In [ ]:
model = keras.Sequential()
model.add(Conv2D(512,(3,3),activation='relu',input_shape = input_size))
model.add(MaxPool2D(2,2))
model.add(Conv2D(512,(3,3),activation='relu'))
model.add(Dropout(0.2))
model.add(MaxPool2D(2,2))
model.add(BatchNormalization())
model.add(Conv2D(512,(3,3),activation='relu'))
model.add(Dropout(0.2))
model.add(MaxPool2D(2,2))
model.add(Conv2D(512,(3,3),activation='relu'))
model.add(MaxPool2D(2,2))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dense(8,activation='softmax'))

In [ ]:
print(model.summary())

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adamax(lr=1e-2), 
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=[tf.keras.metrics.CategoricalAccuracy(name='accuracy'),
                       tf.keras.metrics.CategoricalCrossentropy(name='crossentropy_loss'),
                       ]
              )

In [ ]:
%ls /content/drive/MyDrive/base_model/

In [ ]:
import os
try:
  if (not os.path.exists(base_dir+'model_saves/')):
    os.mkdir(base_dir+'model_saves/')
except FileExistsError:
  print('File Exists')
  

In [ ]:
train_history_list = {'initial':[],'secondary':[]}
val_history_list = {'initial':[],'secondary':[]}


In [ ]:

history = model.fit_generator(
    generator=train_generator, steps_per_epoch=None, epochs=50, verbose=1, callbacks=None,
    validation_data=val_generator,
    shuffle=True, initial_epoch=0)

train_history_list['initial'].append(history)

model.save_weights(model_save+str(times)+'_init.h5')